In [1]:
import qsharp
import qsharp_widgets
import json
from diskcache import Cache
import re_utils

re_utils.DEBUG = True

cache = Cache("~/quant-arith-cache/re-multipliers")
qsharp.init(project_root="../lib/")

@cache.memoize()
def estimate_resources_multiply_pareto(op, n):
    est = qsharp.estimate(f"EstimateUtils.RunMultiply({n},{op})", params={"estimateType": "frontier"})
    return est

ops_and_max_n = [
  ("QuantumArithmetic.JHHA2016.Multiply", "Shift-and-Add (JHHA)", 2**15),
  ("QuantumArithmetic.MCT2017.Multiply", "Shift-And-Add (MCT)", 2**15),
  ("QuantumArithmetic.CG2019.MultiplyKaratsuba32", "Karatsuba", 2**15),
  ("QuantumArithmetic.CG2019.MultiplyKaratsuba", "Karatsuba-8", 27554),
  ("QuantumArithmetic.CG2019.MultiplySchoolbook", "Schoolbook", 11585),
  ("QuantumArithmetic.OFOSG2023.MultiplyWallaceTree", "Wallace Tree", 1024),
]

In [4]:
n = 2**8
results = {} 
i = 0 
for op, _, _ in ops_and_max_n:
    results[i] = estimate_resources_multiply_pareto(op, n)
    i += 1

qsharp_widgets.EstimatesOverview(results, runNames=[name for _, name, _ in ops_and_max_n])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…

The test above was limited to `n=2**8` due to the Wallace Tree multiplier. However, below is a plot of the results of `n=2**10+1` without the Wallace Tree multiplier.

In [3]:
n = 2**10+1 #add 1 to stop Wallace Tree multiplier from running
results = {} 
i = 0 
for op, _, max_n in ops_and_max_n:
    if max_n >= n:
        results[i] = estimate_resources_multiply_pareto(op, n)
        i += 1

qsharp_widgets.EstimatesOverview(results, runNames=[name for _, name, max_n in ops_and_max_n if max_n >= n])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…